In [6]:
# load libraries
import json 



In [7]:
# load raw data 
# read 'papers-with-abstracts (1).json'

with open('../data/raw/papers-with-abstracts (1).json', 'r') as f:
    all_papers = json.load(f)

# read evaluation-tables.json
with open('../data/raw/evaluation-tables.json', 'r') as f:
    evaluation_tables = json.load(f)
    
# read links-between-papers-and-code.json
with open('../data/raw/links-between-papers-and-code.json', 'r') as f:
    links = json.load(f)

# read methods (1).json
with open('../data/raw/methods (1).json', 'r') as f:
    methods = json.load(f)

In [9]:

# filter papers with tasks
def is_paper_valid(paper):
    if paper['tasks']==[]:
        return False
    if paper['authors']==[]:
        return False
    return True
all_papers = [paper for paper in all_papers if is_paper_valid(paper)]

In [10]:
# transform to pandas dataframe
import pandas as pd
df = pd.DataFrame(all_papers)


In [11]:
# transform publish_date to datetime
import datetime
def intify(x):
    try : 
        return int(x)
    except:
        return 0
df['date_year'] = df['date'].apply(lambda x: intify(x.split('-')[0]))


In [12]:
def get_paper_eval(eval_row):
    nodes = []
    # get task
    task = eval_row['task']
    for dataset in eval_row['datasets']:
        dataset_name = dataset['dataset']
        sotas = dataset['sota']['rows']
        for sota in sotas:
            paper_url = sota['paper_url']
            model_name = sota['model_name']
            metrics = sota['metrics']
            node = {
                'task': task,
                'dataset': dataset_name,
                'model_name': model_name,
                'paper_url': paper_url,
                'metrics': metrics
            }
            nodes.append(node)
    return nodes


In [13]:
evals =[]
for eval_row in evaluation_tables:
    nodes = get_paper_eval(eval_row)
    evals.extend(nodes)

In [14]:
# transform to pandas dataframe
df_eval = pd.DataFrame(evals)


In [15]:
# transform to pandas dataframe
df_links = pd.DataFrame(links)


In [16]:
# transform to pandas dataframe
df_methods = pd.DataFrame(methods)

In [18]:
# save to csv
df_eval.to_csv('../data/bronze/evaluation.csv', index=False)
df_links.to_csv('../data/bronze/codes.csv', index=False)
df_methods.to_csv('../data/bronze/methods.csv', index=False)
df.to_csv('../data/bronze/papers.csv', index=False)
